In [1]:
import os
os.chdir(os.path.abspath(".."))


import librosa
import pandas as pd
import numpy as np
import warnings
import time
warnings.filterwarnings('ignore')

from src.dataload import load_audio_data, audio_select_mean_vote
from src.emotion_translation import ekman_emotion
from src.metricas import classical_features, get_training_features_simple

In [2]:
df_annotations = pd.read_excel("data/annotations_2.xlsx")
df_training_audios = df_annotations[df_annotations['Type'] == 'Train'][['Audio_Name','Part_Num']].drop_duplicates()

In [3]:
start_time = time.time()
frame_duration = 2.5
start = 0
step = 0.5

# Main loop for audio data
for index, row in df_training_audios.iterrows():
    
    # Input
    data, timestamps, sr = load_audio_data(df_annotations, part_num = row['Part_Num'], audio_name = row['Audio_Name'])
    df_input = pd.DataFrame(data = np.stack([timestamps, data]).T, columns = ['Time','Data'])
    
    # Labels
    df_label = audio_select_mean_vote(df_annotations,  part_num = row['Part_Num'], audio_name = row['Audio_Name'])
    df_label['Emotion'] = df_label.apply(lambda row : ekman_emotion(row['Valence'], row['Arousal'], row['Dominance']), axis = 1)
    
    time_index = pd.concat([df_label['Time'],df_input['Time']], ignore_index = True).drop_duplicates().sort_values().reset_index(drop = True)
    df_train = pd.merge(time_index, df_input, how = 'left', on = 'Time')
    df_train = pd.merge(df_train, df_label, how = 'left', on = 'Time')
    
    df_train = df_train.fillna(method='ffill').fillna(method='bfill')
    df_train = df_train.set_index('Time').drop_duplicates().reset_index()
    
    paramaters = {'sr' : sr}
    df_features = get_training_features_simple(classical_features, paramaters, frame_duration, start, step, df_train)
    
    break

end_time = time.time()
print(end_time - start_time)    

In [7]:
df_train

,Time,Data,Arousal,Dominance,Valence,Emotion
0,0.000000,0.052490,-0.002000,-0.001429,0.000000,disgust
1,0.000063,-0.003693,-0.002000,-0.001429,0.000000,disgust
2,0.000125,-0.025360,-0.002000,-0.001429,0.000000,disgust
3,0.000188,0.025085,-0.002000,-0.001429,0.000000,disgust
4,0.000250,0.036713,-0.002000,-0.001429,0.000000,disgust
...,...,...,...,...,...,...
4274081,306.087700,-0.001862,27.528333,34.624286,12.072857,joy
4274082,306.099400,-0.001862,27.528333,34.624286,12.241429,joy
4274083,306.110200,-0.001862,27.268333,34.624286,12.241429,joy
4274084,306.115000,-0.001862,27.268333,34.958571,12.241429,joy


In [6]:
df_features

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,labels
0,0.129173,0.63348,0.637506,0.629961,0.618175,0.614299,0.64168,0.683082,0.669334,0.663436,...,0.007464,0.00629,0.00585,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,surprise
1,0.129173,0.63348,0.637506,0.629961,0.618175,0.614299,0.64168,0.683082,0.669334,0.663436,...,0.007464,0.00629,0.00585,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,surprise
2,0.129173,0.63348,0.637506,0.629961,0.618175,0.614299,0.64168,0.683082,0.669334,0.663436,...,0.007464,0.00629,0.00585,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,joy
3,0.129173,0.63348,0.637506,0.629961,0.618175,0.614299,0.64168,0.683082,0.669334,0.663436,...,0.007464,0.00629,0.00585,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,joy
4,0.129173,0.63348,0.637506,0.629961,0.618175,0.614299,0.64168,0.683082,0.669334,0.663436,...,0.007464,0.00629,0.00585,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,joy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,0.129173,0.63348,0.637506,0.629961,0.618175,0.614299,0.64168,0.683082,0.669334,0.663436,...,0.007464,0.00629,0.00585,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,joy
604,0.129173,0.63348,0.637506,0.629961,0.618175,0.614299,0.64168,0.683082,0.669334,0.663436,...,0.007464,0.00629,0.00585,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,joy
605,0.129173,0.63348,0.637506,0.629961,0.618175,0.614299,0.64168,0.683082,0.669334,0.663436,...,0.007464,0.00629,0.00585,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,joy
606,0.129173,0.63348,0.637506,0.629961,0.618175,0.614299,0.64168,0.683082,0.669334,0.663436,...,0.007464,0.00629,0.00585,0.005232,0.004392,0.004108,0.003518,0.002717,0.001884,joy
